In [ ]:
!pip install --upgrade gensim -q
!pip install matplotlib -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 19.1 MB/s eta 0:00:00


In [ ]:
drivePath = '/content/drive'
from google.colab import drive
drive.mount(drivePath, force_remount=True)

Mounted at /content/drive


In [ ]:
import sys
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_validate
sys.path.append(drivePath + '/MyDrive')

import re
import string
import os,glob
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import keras
from nltk.corpus import stopwords
from ml_crossclassifiers.classifiers import Classifiers
from gensim.models import KeyedVectors

In [ ]:
import os
import pandas as pd

# Define paths to the folders
base_path = "/content/drive/MyDrive/PhD Dataset/Sentence_level"  # Change this to your dataset folder
non_plagiarized_path = os.path.join(base_path, "Non_plagiarized")
plagiarized_path = os.path.join(base_path, "Plagiarized")

# Function to read text files, skipping the first line
def load_data(folder_path, label):
    data = []
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        with open(file_path, "r", encoding="utf-8") as f:
            lines = f.readlines()  # Read all lines
            text = " ".join(lines[1:]).strip()  # Skip the first line and join remaining
            data.append((text, label))
    return data

# Load data from both folders
non_plagiarized_data = load_data(non_plagiarized_path, 0)  # Label 0 for Non-plagiarized
plagiarized_data = load_data(plagiarized_path, 1)  # Label 1 for Plagiarized

# Ensure both lists are of equal length
min_length = min(len(non_plagiarized_data), len(plagiarized_data))
non_plagiarized_data = non_plagiarized_data[:min_length]
plagiarized_data = plagiarized_data[:min_length]

# Interleave the data (Plagiarized first, then Non-plagiarized)
interleaved_data = []
for i in range(min_length):
    interleaved_data.append(plagiarized_data[i])  # Label 1
    interleaved_data.append(non_plagiarized_data[i])  # Label 0

# Convert to DataFrame
df = pd.DataFrame(interleaved_data, columns=["Text", "Label"])

# Display dataset info
print(df.head(10))  # Show first 10 rows
print(f"Total samples: {len(df)}")
print(df['Label'].value_counts())

# Save to CSV for further processing
df.to_csv("plagiarism_dataset_interleaved.csv", index=False)
df

                                                Text  Label
0  اس دنیا کے معاشرہ کی اصلاح وفلاح دراصل باہمی ح...      1
1  پیپلزپارٹی کے چیئرپرسن بلاول زرداری بقرعید پر ...      0
2  اللہ رب العزت نے انسانوں کو مختلف رشتوں میں پر...      1
3  اقبال نے کہا تھا کہ ’’ہیں تلخ بہت بندہ مزدور ک...      0
4  حضرت ابوہریرہ ؓ کہتے ہیں کہ ایک شخص نے عرض کیا...      1
5  دنیا بھر میں انتخابی عمل، جمہوریت اور جمہوری ع...      0
6  اللہ رب العزت نے انسانوں کو مختلف رشتوں میں پر...      1
7  شاید مجھے زندگی میںکبھی اتنی خوشی نہ ملی ہو جت...      0
8  حقوق العباد میں سب سے مقدم حق والدین کا ہے۔ ان...      1
9  ویسے تو ملک کے کسی بھی حصے میں امن وامان کی صو...      0
Total samples: 5436
Label
1    2718
0    2718
Name: count, dtype: int64


,Text,Label
0,اس دنیا کے معاشرہ کی اصلاح وفلاح دراصل باہمی ح...,1
1,پیپلزپارٹی کے چیئرپرسن بلاول زرداری بقرعید پر ...,0
2,اللہ رب العزت نے انسانوں کو مختلف رشتوں میں پر...,1
3,اقبال نے کہا تھا کہ ’’ہیں تلخ بہت بندہ مزدور ک...,0
4,حضرت ابوہریرہ ؓ کہتے ہیں کہ ایک شخص نے عرض کیا...,1
...,...,...
5431,گزشتہ اڑتالیس دن سے امریکیوں نے شمالی وزیرستان...,0
5432,یہ خوبصورت بات میں نے اوائل عمری میں ہی اپنی ن...,1
5433,اب جب کہ ایک بنکار، ذکااشرف، ایک ٹی وی مبصر او...,0
5434,یہ خوبصورت بات میں نے اوائل عمری میں ہی اپنی ن...,1


In [ ]:
df['Text'] = df['Text'].apply(lambda x: " ".join(x.lower() for x in x.split())) #convert text into lower case

df['Text'] = df['Text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])    # Remove URLs

df['Text'] = df['Text'].map(lambda x: re.sub(r'\W+', ' ', x)) # Remove Punctuation marks


print("\nDataset After Text Preprocessing:")
print("=================================================\n")
df.head(7)


Dataset After Text Preprocessing:



,Text,Label
0,اس دنیا کے معاشرہ کی اصلاح وفلاح دراصل باہمی ح...,1
1,پیپلزپارٹی کے چیئرپرسن بلاول زرداری بقرعید پر ...,0
2,اللہ رب العزت نے انسانوں کو مختلف رشتوں میں پر...,1
3,اقبال نے کہا تھا کہ ہیں تلخ بہت بندہ مزدور کے ...,0
4,حضرت ابوہریرہ کہتے ہیں کہ ایک شخص نے عرض کیا ی...,1
5,دنیا بھر میں انتخابی عمل جمہوریت اور جمہوری عم...,0
6,اللہ رب العزت نے انسانوں کو مختلف رشتوں میں پر...,1


In [ ]:
df.head(7)

,Text,Label
0,اس دنیا کے معاشرہ کی اصلاح وفلاح دراصل باہمی ح...,1
1,پیپلزپارٹی کے چیئرپرسن بلاول زرداری بقرعید پر ...,0
2,اللہ رب العزت نے انسانوں کو مختلف رشتوں میں پر...,1
3,اقبال نے کہا تھا کہ ہیں تلخ بہت بندہ مزدور کے ...,0
4,حضرت ابوہریرہ کہتے ہیں کہ ایک شخص نے عرض کیا ی...,1
5,دنیا بھر میں انتخابی عمل جمہوریت اور جمہوری عم...,0
6,اللہ رب العزت نے انسانوں کو مختلف رشتوں میں پر...,1


In [ ]:
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import spacy
import string

In [ ]:
gensim.__version__

'4.3.3'

In [ ]:
np.random.seed(42)

In [ ]:
import gensim.downloader as api
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [ ]:
from gensim.models import KeyedVectors
wv = KeyedVectors.load('/content/drive/MyDrive/Embeddings/fasttext-wiki-news-subwords-300.kv')

In [ ]:
def sent_vec(sent):
    vector_size = wv.vector_size
    wv_res = np.zeros(vector_size)
    # print(wv_res)
    ctr = 1
    for w in sent:
        if w in wv:
            ctr += 1
            wv_res += wv[w]
    wv_res = wv_res/ctr
    return wv_res

In [ ]:
df['vec'] = df['Text'].apply(sent_vec)

In [ ]:
df.head()

,Text,Label,vec
0,اس دنیا کے معاشرہ کی اصلاح وفلاح دراصل باہمی ح...,1,"[-0.06149829254531314, 0.011366866461801862, -..."
1,پیپلزپارٹی کے چیئرپرسن بلاول زرداری بقرعید پر ...,0,"[-0.05959524100528084, 0.011008667655463092, -..."
2,اللہ رب العزت نے انسانوں کو مختلف رشتوں میں پر...,1,"[-0.058019690281631425, 0.01033555288004655, -..."
3,اقبال نے کہا تھا کہ ہیں تلخ بہت بندہ مزدور کے ...,0,"[-0.06042801416176173, 0.01003752258799955, -0..."
4,حضرت ابوہریرہ کہتے ہیں کہ ایک شخص نے عرض کیا ی...,1,"[-0.06041129267091615, 0.010786577788466884, -..."


In [ ]:
X = df['vec'].to_list()
y = df['Label'].to_list()

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier_LR = LogisticRegression()

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

# Assuming X and y are your feature matrix and target vector respectively


# Initialize StratifiedKFold with 5 folds
skf = StratifiedKFold(n_splits=5)

In [ ]:
# Initialize classifiers
logreg = LogisticRegression()
dtree = DecisionTreeClassifier()
rf = RandomForestClassifier()
svm = SVC()
knn = KNeighborsClassifier()
nb = GaussianNB()
lda = LinearDiscriminantAnalysis()

xgb = GradientBoostingClassifier()

In [ ]:
# Create a voting classifier
voting_clf = VotingClassifier(estimators=[
    ('logreg', logreg),
    ('dtree', dtree),
    ('rf', rf),
    ('svm', svm),
    ('knn', knn),
    ('nb', nb),
    ('lda', lda),
    ('xgb', xgb)
], voting='hard')

In [ ]:
# Initialize classifiers list
classifiers = [logreg, dtree, rf, svm, knn, nb, lda, xgb, voting_clf]

In [ ]:
from sklearn.model_selection import cross_val_score, KFold

In [ ]:
n_folds = 5

# Initialize KFold object
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_score

original_mean = Classifiers()

# Iterate over classifiers
for clf in classifiers:
    clf_name = clf.__class__.__name__  # Get classifier name

    scores_accuracy = cross_val_score(clf, X, y, cv=skf, scoring='accuracy')
    scores_recall = cross_val_score(clf, X, y, cv=skf, scoring='recall')
    scores_precision = cross_val_score(clf, X, y, cv=skf, scoring='precision')


    # Now np.mean automatically filters based on classifier name
    mean_accuracy = original_mean.compute_mean(scores_accuracy, clf_name, metric_name="accuracy")
    mean_recall = original_mean.compute_mean(scores_recall, classifier_name=clf_name, metric_name="recall")
    mean_precision = original_mean.compute_mean(scores_precision, classifier_name=clf_name, metric_name="precision")


    # Print metrics
    print(f"Metrics for {clf_name}:")
    print(f"**Accuracy:** {mean_accuracy:.4f}")
    print(f"**Recall:** {mean_recall:.4f}")
    print(f"**Precision:** {mean_precision:.4f}")


Metrics for LogisticRegression:
**Accuracy:** 0.8936
**Recall:** 0.8577
**Precision:** 0.8961
Metrics for DecisionTreeClassifier:
**Accuracy:** 0.8214
**Recall:** 0.8261
**Precision:** 0.8276
Metrics for RandomForestClassifier:
**Accuracy:** 0.8920
**Recall:** 0.8924
**Precision:** 0.8460


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Metrics for SVC:
**Accuracy:** 0.8966
**Recall:** 0.8904
**Precision:** 0.8926
Metrics for KNeighborsClassifier:
**Accuracy:** 0.8861
**Recall:** 0.8960
**Precision:** 0.8355
Metrics for GaussianNB:
**Accuracy:** 0.8759
**Recall:** 0.8648
**Precision:** 0.8728
Metrics for LinearDiscriminantAnalysis:
**Accuracy:** 0.8834
**Recall:** 0.8523
**Precision:** 0.8674
Metrics for GradientBoostingClassifier:


TypeError: unsupported format string passed to NoneType.__format__